This will be my GAN model used to create semi-realistic face images using the celeb A dataset.  First I will import all of the relevant libraries 

In [ ]:
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from torchvision import datasets

Next I will create a seed for this torch model in order to have reproducibility for the model.  We will also use deterministic algorithms to true as well to again ensure reproducibility of the results. 

In [ ]:
manualSeed = 999 # Random choice for reproducing same results
print("Random Seed: ", manualSeed) # Print out the seed 
random.seed(manualSeed) # Set the seed 
torch.manual_seed(manualSeed) # Set the seed in pytorch
torch.use_deterministic_algorithms(True) # Turn on the deterministic algorithms 

Next we will setup the hyperparameters for the model.  These include the dataset directory path, the number of workers for the dataloader, the batch size for training, the size of the images.  We also set the number of input channels for the model (3 for colour images), the latent vector size for the generator input, the feature map sizes for the generator and the discriminator.  The number of epochs or training cycles, the learning rate of the model, the Beta1 paramentere for the Adam optimiser and the number of available GPU's 

In [ ]:
dataroot = './CelebA' # Root directory for dataset
workers = 2 # Number of workers for dataloader
batch_size = 128 # Batch size during training
image_size = 64 # Spatial size of training images. All images will be resized to this. #   size using a transformer.
nc = 3 # Number of channels in the training images. For color images this is 3
nz = 100 # Size of z latent vector (i.e. size of generator input)
ngf = 64 # Size of feature maps in generator
ndf = 64 # Size of feature maps in discriminator
num_epochs = 5 # Number of training epochs, arbitrary choice mainly for time 
lr = 0.0002 # Learning rate for optimizers, we want to avoid overshooting 
beta1 = 0.5 # Beta1 hyperparameter for Adam optimizers
ngpu = 1 # Number of GPUs available. Use 0 for CPU mode.